# Introduction

This article is part of the final homework for Data Anlysis & Geometry. The article (I think) is also designed to answer the questions that audience have due to shortage of the speech time. The article is not intended to be a comprehensive review of the fields of Generative model. Many excellent papers and algorithm is ignored in the article due to the limitation of the length of an article.The generative model is really an interesting reserach aspect.


 The article is intended to describes:
 three parts of generative models are introduced in this article：
 1.  Three important models in generative models fields, they are: VAE(variational Autoencoder), vanilla GAN(generative adversarial network), and the improved version of vanilla gan:WGAN(Wasserstein GAN)
 2.  Some critical mathematical inductions to reveal the key ideas of them
 3.  Some advantages & disadvantages they have respectively
 4.  Experimental results in mnist character generation


# General Principle

These three models, they all work via the principle of maximum likelihood. But VAE work with MLE directly and GAN series work on it indirectly(by sampling from $p_{data}$), by focusing on maximum likelihood, we can eliminate some of the more distracting differences between these three models

The basic idea of maximum likelihood is to define a model that provides an estimate of a probability distribution, parameterized by parameters $\theta$. We then refer to the likelihood as the probability that the model assigns to the trainingdata:$\prod_{i=1}^{m}p_{model}(x(i);\theta)$,for a dataset containing $m$ training examples.

The principle of maximum likelihood simply says to choose the parametersfor the model that maximize the likelihood of the training data.  This is easiestto do in log space, where we have a sum rather than a product over examples.

<center>$\theta^{*} = \arg\max_\theta \prod_{i=1}^m p_\theta(\mathbf{x}^{(i)})$</center>
<center>$\theta^{*} = \arg\max_\theta \sum_{i=1}^n \log p_\theta(\mathbf{x}^{(i)})$<center>
    
Actually, We can also think of maximum likelihood estimation as minimizing the KL divergence between the data generating distribution and the model
    
<center>$\boldsymbol{\theta}^{*}=\underset{\boldsymbol{\theta}}{\arg \min } D_{\mathrm{KL}}\left(p_{\text {data }}(\boldsymbol{x}) \| p_{\text {model }}(\boldsymbol{x} ; \boldsymbol{\theta})\right)$<center>
    
The Generative model working via maximun likelihood have several variants, judging from the relationship with the data distribution and the tractablity of the density function, which is illustrated in the following picture：

<img src="Taxonomy%20of%20Generative%20Models.png" width="450"/>

And we will focus on VAE in Explicit models requiring approximation and GAN series in Implicit density models

# Variational Autoencoder

## Critical Idea
 1. VAE is a explicit density model

Explicit density model means a definition of an explicit density function $p_{model}(x(i);\theta)$, these kind of models' optimization can be straightforward, by taking the gradient uphill algorithm ,by NFL(No Free Lunch) Principle, we find explicit density models with tractable function have some problems. 

To avoid the problems of explicit density models with tractable function, intractable function is introduced to be the density function. Meanwhile, special optimization like variational approximations is used to treat the problem.

 2. VAE is a variant of Autoencoder architecture
 
Unlike classical (sparse, denoising, etc.) autoencoders, Variational autoencoders (VAEs) are generative models, like Generative Adversarial Networks. Their association with this group of models derives mainly from the architectural affinity with the basic autoencoder (the final training objective has an encoder and a decoder), but their mathematical formulation differs significantly. VAEs are directed probabilistic graphical models (DPGM) whose posterior is approximated by a neural network, forming an autoencoder-like architecture.

In other words, VAE has a architecture of Autoencoder, which set a latent space and try to minimize the distance between input and output, but unlike the other Autoencoders, it use probalistic method to treat latent space instead of setting a static latent space which is remained to be mined by dimension reduction map

## VAE Model

1.Treating latent vector as a distribution, Making some notation:
- Prior $p_{\theta}(z)$
- Likelihood $p_{\theta}(x|z)$
- Posterior $p_{\theta}(z|x)$

Assuming that we know the real parameter $\theta^{*}$ for this distribution. In order to generate a sample that looks like a real data point $x^{(i)}$, we follow these steps:
1. sample a $z^{(i)}$ from a prior distribution $p_{\theta^*}(\mathbf{z})$.
2. value $x^{(i)}$ is generated from a conditional distribution $p_{\theta^*}(\mathbf{x} \vert \mathbf{z} = \mathbf{z}^{(i)})$

where $ \theta^{*} = \arg\max_\theta \sum_{i=1}^n \log p_\theta(\mathbf{x}^{(i)})$

Then we get $p_\theta(\mathbf{x}^{(i)}) = \int p_\theta(\mathbf{x}^{(i)}\vert\mathbf{z}) p_\theta(\mathbf{z}) d\mathbf{z}$

Unlike usual treatment like SGD, we found it expensive to compute since every $z$ will be checked and sum them up. To narrow down the serach space, approxmation function to output $q_\phi(\mathbf{z}\vert\mathbf{x})$ is introduced here:


$
\begin{aligned}
& D_\text{KL}( q_\phi(\mathbf{z}\vert\mathbf{x}) \| p_\theta(\mathbf{z}\vert\mathbf{x}) ) & \\
&=\int q_\phi(\mathbf{z} \vert \mathbf{x}) \log\frac{q_\phi(\mathbf{z} \vert \mathbf{x})}{p_\theta(\mathbf{z} \vert \mathbf{x})} d\mathbf{z} & \\
&=\int q_\phi(\mathbf{z} \vert \mathbf{x}) \log\frac{q_\phi(\mathbf{z} \vert \mathbf{x})p_\theta(\mathbf{x})}{p_\theta(\mathbf{z}, \mathbf{x})} d\mathbf{z} & \scriptstyle{\text{; Because }p(z \vert x) = p(z, x) / p(x)} \\
&=\int q_\phi(\mathbf{z} \vert \mathbf{x}) \big( \log p_\theta(\mathbf{x}) + \log\frac{q_\phi(\mathbf{z} \vert \mathbf{x})}{p_\theta(\mathbf{z}, \mathbf{x})} \big) d\mathbf{z} & \\
&=\log p_\theta(\mathbf{x}) + \int q_\phi(\mathbf{z} \vert \mathbf{x})\log\frac{q_\phi(\mathbf{z} \vert \mathbf{x})}{p_\theta(\mathbf{z}, \mathbf{x})} d\mathbf{z} & \scriptstyle{\text{; Because }\int q(z \vert x) dz = 1}\\
&=\log p_\theta(\mathbf{x}) + \int q_\phi(\mathbf{z} \vert \mathbf{x})\log\frac{q_\phi(\mathbf{z} \vert \mathbf{x})}{p_\theta(\mathbf{x}\vert\mathbf{z})p_\theta(\mathbf{z})} d\mathbf{z} & \scriptstyle{\text{; Because }p(z, x) = p(x \vert z) p(z)} \\
&=\log p_\theta(\mathbf{x}) + \mathbb{E}_{\mathbf{z}\sim q_\phi(\mathbf{z} \vert \mathbf{x})}[\log \frac{q_\phi(\mathbf{z} \vert \mathbf{x})}{p_\theta(\mathbf{z})} - \log p_\theta(\mathbf{x} \vert \mathbf{z})] &\\
&=\log p_\theta(\mathbf{x}) + D_\text{KL}(q_\phi(\mathbf{z}\vert\mathbf{x}) \| p_\theta(\mathbf{z})) - \mathbb{E}_{\mathbf{z}\sim q_\phi(\mathbf{z}\vert\mathbf{x})}\log p_\theta(\mathbf{x}\vert\mathbf{z}) &
\end{aligned} 
$

# Generative Adversarial Network

# Wasserstein GAN

 <img src="EM%20distance%20illustration.png" width="450"/>

In [ ]:
https://lilianweng.github.io/lil-log/2018/08/12/from-autoencoder-to-beta-vae.html

In [ ]:
NIPS 2016 Tutorial:Generative Adversarial Networks Ian Goodfellow

In [ ]:
https://en.wikipedia.org/wiki/Autoencoder